In [1]:
# offline use of TabPFN see https://www.kaggle.com/datasets/neutrino404/tabpfn
# add data source https://www.kaggle.com/datasets/neutrino404/tabpfn
!pip install tabpfn --no-index --find-links=file:///kaggle/input/tabpfn

Looking in links: file:///kaggle/input/tabpfn
Processing /kaggle/input/tabpfn/tabpfn-0.1.9-py3-none-any.whl


In [2]:
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/tabpfn/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

In [3]:
import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os 
#from matplotlib import pyplot as plt
#from sklearn.tree import DecisionTreeClassifier
#from sklearn import tree
#from sklearn.impute import SimpleImputer
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import RepeatedStratifiedKFold
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
#from sklearn.ensemble import RandomForestClassifier
#from catboost import CatBoostClassifier
#from sklearn.model_selection import GridSearchCV
#from sklearn.neural_network import MLPClassifier
#import seaborn as sns
from sklearn.metrics import make_scorer
from sklearn.metrics import log_loss
from tabpfn import TabPFNClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.impute import KNNImputer

for dirname, _, filenames in os.walk('/kaggle/input'): 
    for filename in filenames: 
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/tabpfn/tabpfn-0.1.9-py3-none-any.whl
/kaggle/input/tabpfn/prior_diff_real_checkpoint_n_0_epoch_100.cpkt
/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [4]:
input_path = "/kaggle/input/icr-identify-age-related-conditions/"
df_train_file = pd.read_csv(input_path + 'train.csv')
#df_greek_file = pd.read_csv(input_path + 'greeks.csv')

In [5]:
# from https://www.kaggle.com/code/datafan07/icr-simple-eda-baseline
# see also https://www.kaggle.com/code/dan3dewey/icr-2023-balanced-log-loss/notebook
# see also https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/409691

def balance_loglossv2(y_true, y_pred):
    target_mean = y_true.mean()
    w0 = 1/(1-target_mean)
    w1 = 1/target_mean
    sample_weight = [w0 if y == 0 else w1 for y in y_true]
    loss = log_loss(y_true, y_pred, sample_weight=sample_weight)
    
    return loss

In [6]:
def prepare_initial(df):
    df.sort_values(by=['Id'], axis=0, ascending=True, inplace=True)
    
    # training set labels
    df_y = None
    if 'Class' in df.columns:
      df_y = df['Class']
    
    # training set input
    best_columns = ['AB', 'DI', 'DH', 'GL', 'DE', 'DA', 'EB', 'CR', 'EE', 'EH', 'FD ', 'CC', 'DL', 'FE', 'DU', 'AF', 'FI', 'BC', 'FL', 
                    'FR', 'BR', 'AM']
    df_input = df.loc[:, best_columns]
    
    return df_input, df_y

In [7]:
cv = StratifiedKFold(n_splits=5)
competition_balanced_log_loss = make_scorer(balance_loglossv2, needs_proba=True, greater_is_better=False)
scores = []

def compute_scores(clf, cv_strat, df_x, df_y, col_name):
    for i in ['accuracy', 'neg_log_loss', competition_balanced_log_loss]:
        scores.append([col_name, i, cross_val_score(clf, df_x, df_y, cv=cv_strat, scoring=i).mean()])
    return scores

In [8]:
df_ready_x, df_ready_y = prepare_initial(df_train_file)

[['TabPFN', 'accuracy', 0.9222659323367427],
 ['TabPFN', 'neg_log_loss', -0.22899856204625743],
 ['TabPFN',
  make_scorer(balance_loglossv2, greater_is_better=False, needs_proba=True),
  -0.500252392413224]]

In [9]:
smote_params = {'k_neighbors': 7, 'sampling_strategy': 1}
#xgb_params = {'learning_rate': 0.15, 'max_depth': 3, 'n_estimators': 38}
imputer_params = {'n_neighbors': 2, 'weights': "uniform", 'missing_values': np.nan, 'copy': False}
tabpfn_params = {'N_ensemble_configurations': 28}
#("c", SMOTE(**smote_params)),
clf = imbpipeline(steps=[("a", KNNImputer(**imputer_params)), ("b", RobustScaler()), ("c", SMOTE(**smote_params)), ("d", TabPFNClassifier(**tabpfn_params))])
clf.fit(df_ready_x, df_ready_y)

compute_scores(clf, cv, df_ready_x, df_ready_y, 'TabPFN')
scores

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Tr

[['TabPFN', 'accuracy', 0.9254917387883556],
 ['TabPFN', 'neg_log_loss', -0.2259256981007224],
 ['TabPFN',
  make_scorer(balance_loglossv2, greater_is_better=False, needs_proba=True),
  -0.5022151031868629]]

### TabPFN with boruta features -0.404532823555921 {N_ensemble_configurations = 48}
### TabPFN with boruta features -0.40205373503904046 {N_ensemble_configurations = 40}
### TabPFN with boruta features -0.39909391870738464 {N_ensemble_configurations = 32}
### TabPFN with boruta features -0.39988480377233665 {N_ensemble_configurations = 30}
### ==>> TabPFN with boruta features -0.3981761378086553 {N_ensemble_configurations = 28}
### TabPFN with boruta features -0.3991469795332882 {N_ensemble_configurations = 27}
### TabPFN with boruta features -0.39824621370886293 {N_ensemble_configurations = 26}
### TabPFN with boruta features -0.39955274422044185 {N_ensemble_configurations = 24}
### TabPFN with boruta features -0.40413397080960073 {N_ensemble_configurations = 16}
### TabPFN with boruta features -0.40810746 {N_ensemble_configurations = 4}
### TabPFN with all features except categorical -0.4294606318709975 {N_ensemble_configurations = 4}


In [10]:
df_test_file = pd.read_csv(input_path + 'test.csv')
df_test_x, df_test_y = prepare_initial(df_test_file)
df_test_preds = pd.DataFrame(clf.predict_proba(df_test_x))

In [11]:
df_test_y = pd.concat([df_test_file['Id'], df_test_preds], axis=1)
df_test_y

,Id,0,1
0,00eed32682bb,0.751172,0.248828
1,010ebe33f668,0.751172,0.248828
2,02fa521e1838,0.751172,0.248828
3,040e15f562a2,0.751172,0.248828
4,046e85c7cc7f,0.751172,0.248828


In [12]:
df_test_y.to_csv('/kaggle/working/submission.csv', header=['Id', 'class_0', 'class_1'], index=False)

In [13]:
!head /kaggle/working/submission.csv

Id,class_0,class_1
00eed32682bb,0.7511716,0.24882847
010ebe33f668,0.7511716,0.24882847
02fa521e1838,0.7511716,0.24882847
040e15f562a2,0.7511716,0.24882847
046e85c7cc7f,0.7511716,0.24882847


## version 7 scored **0.25** with TabPFN and Boruta features
## version 9 scored **0.29** with TabPFN and all non categorical features